In [1]:
from copy import deepcopy
import os
import multiprocessing as mp

import gsd.hoomd
import mbuild as mb
import numpy as np
import gsd.pygsd
from morphct import execute_qcc as eqcc
from morphct import mobility_kmc as kmc
from morphct import chromophores
from morphct import kmc_analyze
from morphct.chromophores import conversion_dict
from morphct.chromophores import amber_dict

def visualize_qcc_input(qcc_input):
    """
    Visualize a quantum chemical input string (for pyscf) using mbuild.
    
    Parameters
    ----------
    qcc_input : str
        Input string to visualize
    """
    comp = mb.Compound()
    for line in qcc_input.split(";")[:-1]:
        atom, x, y, z = line.split()
        xyz = np.array([x,y,z], dtype=float)
        # Angstrom -> nm
        xyz /= 10
        comp.add(mb.Particle(name=atom,pos=xyz))
    comp.visualize().show()
    
def from_snapshot(snapshot, scale=1.0):
    """
    Convert a hoomd.data.Snapshot or a gsd.hoomd.Snapshot to an
    mbuild Compound.
    
    Parameters
    ----------
    snapshot : hoomd.data.SnapshotParticleData or gsd.hoomd.Snapshot
        Snapshot from which to build the mbuild Compound.
    scale : float, optional, default 1.0
        Value by which to scale the length values
        
    Returns
    -------
    comp : mb.Compound
    """
    comp = mb.Compound()
    bond_array = snapshot.bonds.group
    n_atoms = snapshot.particles.N

    # There will be a better way to do this once box overhaul merged
    try:
        # gsd
        box = snapshot.configuration.box
        comp.box = mb.box.Box(lengths=box[:3] * scale)
    except AttributeError:
        # hoomd
        box = snapshot.box
        comp.box = mb.box.Box(lengths=np.array([box.Lx,box.Ly,box.Lz]) * scale)

    # to_hoomdsnapshot shifts the coords, this will keep consistent
    shift = np.array(comp.box.lengths)/2
    # Add particles
    for i in range(n_atoms):
        name = snapshot.particles.types[snapshot.particles.typeid[i]]
        xyz = snapshot.particles.position[i] * scale + shift
        charge = snapshot.particles.charge[i]

        atom = mb.Particle(name=name, pos=xyz, charge=charge)
        comp.add(atom, label=str(i))

    # Add bonds
    particle_dict = {idx: p for idx, p in enumerate(comp.particles())}
    for i in range(bond_array.shape[0]):
        atom1 = int(bond_array[i][0])
        atom2 = int(bond_array[i][1])
        comp.add_bond([particle_dict[atom1], particle_dict[atom2]])
    return comp
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

/Users/jimmy/miniconda3/envs/morphct-ex/lib/python3.7/site-packages/h5py/__init__.py:46: DeprecationWarning: `np.typeDict` is a deprecated alias for `np.sctypeDict`.
  from ._conv import register_converters as _register_converters


ceating a list of transfer integrals. the data set is 100 gsd with 2 thiophene
rings filled randomly in a box of 10^3 angstroms. they arent allowed to be within 2 angstroms of each other.

In [2]:
transfer_integral_list = []
for i in range(100):
    gsd_file = "/Users/jimmy/repos/morphct-jimmy/examples/thiothene-gsds/thiothene"+str(i)+".gsd"
    with gsd.hoomd.open(name=gsd_file, mode='rb') as f:
        snap = f[0]
    
        
    ref_distance = 1
    
    snap.particles.position *= ref_distance
    snap.configuration.box[:3] *= ref_distance
    print(snap.configuration.box)
    
    chromo_ids = np.array([0,1,2,3,4,5,6,7,8,])
    chromo2_ids = np.array([9,10,11,12,13,14,15,16,17])
    aaids=[]
    aaids.append(chromo_ids)
    aaids.append(chromo2_ids)
    chromo_list = []
    for i,aaid in enumerate(aaids):
        chromo_list.append(chromophores.Chromophore(i, snap, aaid, "donor", amber_dict))
        
    qcc_pairs = chromophores.set_neighbors_voronoi(chromo_list, snap, amber_dict, d_cut=100)
    
    dimer_data = eqcc.dimer_homolumo(qcc_pairs, "two-molecule-test.txt")
    
    data = eqcc.singles_homolumo(chromo_list, "singles.txt")
    
    eqcc.set_energyvalues(chromo_list, "singles.txt", "two-molecule-test.txt")
    chromo =chromo_list[i]
    transfer_integral_list.append(chromo.neighbors_ti[0])

[10. 10. 10.  0.  0.  0.]
[10. 10. 10.  0.  0.  0.]
[10. 10. 10.  0.  0.  0.]
[10. 10. 10.  0.  0.  0.]
[10. 10. 10.  0.  0.  0.]
[10. 10. 10.  0.  0.  0.]
[10. 10. 10.  0.  0.  0.]
[10. 10. 10.  0.  0.  0.]
[10. 10. 10.  0.  0.  0.]
[10. 10. 10.  0.  0.  0.]
[10. 10. 10.  0.  0.  0.]
[10. 10. 10.  0.  0.  0.]
[10. 10. 10.  0.  0.  0.]
[10. 10. 10.  0.  0.  0.]
[10. 10. 10.  0.  0.  0.]
[10. 10. 10.  0.  0.  0.]
[10. 10. 10.  0.  0.  0.]
[10. 10. 10.  0.  0.  0.]
[10. 10. 10.  0.  0.  0.]
[10. 10. 10.  0.  0.  0.]
[10. 10. 10.  0.  0.  0.]
[10. 10. 10.  0.  0.  0.]
[10. 10. 10.  0.  0.  0.]
[10. 10. 10.  0.  0.  0.]
[10. 10. 10.  0.  0.  0.]
[10. 10. 10.  0.  0.  0.]
[10. 10. 10.  0.  0.  0.]
[10. 10. 10.  0.  0.  0.]
[10. 10. 10.  0.  0.  0.]
[10. 10. 10.  0.  0.  0.]
[10. 10. 10.  0.  0.  0.]
[10. 10. 10.  0.  0.  0.]
[10. 10. 10.  0.  0.  0.]
[10. 10. 10.  0.  0.  0.]
[10. 10. 10.  0.  0.  0.]
[10. 10. 10.  0.  0.  0.]
[10. 10. 10.  0.  0.  0.]
[10. 10. 10.  0.  0.  0.]
[10. 10. 10.

In [3]:
print(transfer_integral_list)

[0.1526627010691445, 0.08635785646032404, 0.011744736265238837, 0.02667794599193282, 0.2343291147838988, 0.2729131791956388, 0.008705460957194051, 0.006669042962339214, 0.0032772950633453528, 0.21273735247027734, 0.20842014136014744, 0.19684052743604522, 0.12645669152152453, 0.015495455251871681, 0.05184608344211228, 0.04101788614582071, 0.1478084707923263, 0.12531016331982586, 0.04859359754108807, 0.2559797958196117, 0.3349104284882897, 0.1270103030355956, 0.2985260806533839, 0.01920199125260205, 0.04138332337561695, 0.008260866630863156, 0.006924663642714361, 0.3113127096192211, 0.004938930201347136, 0.4332225141547739, 0.01167924650727386, 0.24050316665225033, 0.08779874234718359, 0.05049278217485004, 0.04333868968509856, 0.33153926200510664, 0.35660277060847245, 0.018857218602945196, 0.7053976988208934, 0.01239868309675748, 0.012732230785814275, 0.019383061455418852, 0.00021270341575610604, 0.016957688793463547, 0.04513125854084401, 0.20530896715511693, 0.18853573393199316, 0.00473

In [4]:
transfer_integrals = np.array(transfer_integral_list)

In [5]:
print(np.average(transfer_integrals))

0.11500427270336701


In [6]:
print(np.min(transfer_integrals),np.max(transfer_integrals))

0.00021270341575610604 0.7053976988208934


In [7]:
print(np.std(transfer_integrals))

0.13715430139293544


In [23]:
visualize_qcc_input(qcc_pairs[0][1])


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [10]:
filepath = "/Users/jimmy/repos/morphct-jimmy/examples/angled_thiophenes_gsds"
transfer_integral_list = []
i=0
for file in os.listdir(filepath):
    gsd_file = "/Users/jimmy/repos/morphct-jimmy/examples/angled_thiophenes_gsds/" + str(file)
    
    with gsd.hoomd.open(name=gsd_file, mode='rb') as f:
        snap = f[0]
           
    ref_distance = 1
    
    snap.particles.position *= ref_distance
    snap.configuration.box[:3] *= ref_distance
    print(snap.configuration.box)
    
    chromo_ids = np.array([0,1,2,3,4,5,6,7,8,])
    chromo2_ids = np.array([9,10,11,12,13,14,15,16,17])
    aaids=[]
    aaids.append(chromo_ids)
    aaids.append(chromo2_ids)
    chromo_list = []
    for i,aaid in enumerate(aaids):
        chromo_list.append(chromophores.Chromophore(i, snap, aaid, "donor", amber_dict))
        
    qcc_pairs = chromophores.set_neighbors_voronoi(chromo_list, snap, amber_dict, d_cut=100)
    
    dimer_data = eqcc.dimer_homolumo(qcc_pairs, "two-molecule-test.txt")
    
    data = eqcc.singles_homolumo(chromo_list, "singles.txt")
    
    eqcc.set_energyvalues(chromo_list, "singles.txt", "two-molecule-test.txt")
    chromo =chromo_list[i]
    transfer_integral_list.append(chromo.neighbors_ti[0])
    i+=1

[10.862557  9.539556  5.018568  0.        0.        0.      ]
[10.851181  9.539394  9.787177  0.        0.        0.      ]
[ 9.550778  9.549399 10.013601  0.        0.        0.      ]
[12.163429  9.539868 10.013601  0.        0.        0.      ]
[10.851181  9.549581 10.014479  0.        0.        0.      ]
[9.663429 9.539394 9.776698 0.       0.       0.      ]
[13.362557  9.539394  5.013601  0.        0.        0.      ]
[13.351181  9.549399 10.013601  0.        0.        0.      ]
[ 9.663429  9.539702 10.018381  0.        0.        0.      ]
[ 9.559577  9.539868 10.013601  0.        0.        0.      ]
[13.351181  9.549399  7.513601  0.        0.        0.      ]
[9.663429 9.539702 7.518381 0.       0.       0.      ]
[9.559577 9.539868 7.513601 0.       0.       0.      ]
[ 9.663429  9.539394 12.276698  0.        0.        0.      ]
[10.862557  9.539394  9.539394  0.        0.        0.      ]
[9.550778 9.549399 7.513601 0.       0.       0.      ]
[12.163429  9.539868  7.513601  

In [12]:
filepath = "/Users/jimmy/repos/morphct-jimmy/examples/angled_thiophenes_gsds"
for file in os.listdir(filepath):
    print(file)

thiophene_d-0.2-0.0_a-0.0-3.1.gsd
thiophene_d-0.2-0.2_a-1.6-1.6.gsd
thiophene_d-0.0-0.5_a-1.6-0.0.gsd
thiophene_d-0.5-0.5_a-3.1-0.0.gsd
thiophene_d-0.2-0.5_a-1.6-3.1.gsd
thiophene_d-0.2-0.2_a-3.1-1.6.gsd
thiophene_d-0.5-0.0_a-0.0-0.0.gsd
thiophene_d-0.5-0.5_a-1.6-0.0.gsd
thiophene_d-0.2-0.5_a-3.1-3.1.gsd
thiophene_d-0.0-0.5_a-3.1-0.0.gsd
thiophene_d-0.5-0.2_a-1.6-0.0.gsd
thiophene_d-0.2-0.2_a-3.1-3.1.gsd
thiophene_d-0.0-0.2_a-3.1-0.0.gsd
thiophene_d-0.2-0.5_a-3.1-1.6.gsd
thiophene_d-0.2-0.0_a-0.0-1.6.gsd
thiophene_d-0.0-0.2_a-1.6-0.0.gsd
thiophene_d-0.5-0.2_a-3.1-0.0.gsd
thiophene_d-0.2-0.2_a-1.6-3.1.gsd
thiophene_d-0.2-0.5_a-1.6-1.6.gsd
thiophene_d-0.5-0.2_a-3.1-1.6.gsd
thiophene_d-0.0-0.2_a-1.6-1.6.gsd
thiophene_d-0.2-0.0_a-0.0-0.0.gsd
thiophene_d-0.5-0.5_a-3.1-3.1.gsd
thiophene_d-0.2-0.5_a-1.6-0.0.gsd
thiophene_d-0.0-0.5_a-1.6-3.1.gsd
thiophene_d-0.0-0.2_a-3.1-1.6.gsd
thiophene_d-0.5-0.0_a-0.0-3.1.gsd
thiophene_d-0.5-0.2_a-1.6-1.6.gsd
thiophene_d-0.0-0.5_a-3.1-3.1.gsd
thiophene_d-0.

In [13]:
print(transfer_integral_list)

[0.5756063730721547, 0.1706274329933303, 0.0004044557503983088, 0.00019542027076680803, 0.0024222841531885645, 0.00698114048205485, 0.06886640409430475, 0.0015794992781640435, 0.0026849216912709925, 0.0033795770487429115, 0.00730340758627097, 0.3881685721598499, 0.021073748494440624, 0.014408285093034346, 0.14120365751744177, 0.08708645516030561, 0.023802883221399, 0.057934908060785856, 0.018026443353434545, 0.07452426433499794, 0.4494855224191888, 0.5741765668862784, 0.0001930314540859358, 0.0023935863507624994, 0.0004181908662250279, 0.3035933212863564, 0.06888018343526892, 0.011478016305053906, 0.0033081267659159403, 0.0015984397502078673, 0.002708262529725187, 0.021037216565173013, 0.02366393095001427, 0.007404832915047468, 0.39339177312358015, 0.0234729754831615, 0.010091746111875416, 0.02349064245096397, 0.05342527670320946, 0.08782892122163428, 0.004591368392899578, 0.0167936913818574, 0.007321705419376019, 0.2429358958487525, 0.020934291489565427, 0.027164689322310972, 0.081634

In [14]:
transfer_integrals = np.array(transfer_integral_list)
print(np.average(transfer_integrals))
print(np.min(transfer_integrals),np.max(transfer_integrals))

0.11036316121729678
0.0001930314540859358 1.1350859337807926


In [18]:
gsd_file = "/Users/jimmy/repos/morphct-jimmy/examples/angled_thiophenes_gsds/thiophene_d-0.5-0.2_a-3.1-3.1.gsd"

with gsd.hoomd.open(name=gsd_file, mode='rb') as f:
    snap = f[0]


In [17]:
print(snap.configuration.box)

[12.163429  9.539702  7.518381  0.        0.        0.      ]


In [19]:
print(np.std(transfer_integrals))

0.18575527179722262
